In [22]:
import pandas as pd
import sqlite3

### Read in the excel file, keep only the required column and put it into a dataframe

In [23]:
# Step 1: Read the Excel file and keep only the specified column
file_path = '../../Group A/data/Theme Parks Survey Responses 4 Nov.csv' 
column_name = """Arrange the following activities in the order in which you experienced them during your visit.
A. Go to the rollercoasters
B. Go to the Water-based rides
C. Go to the F&B outlet
D. Go and shop for souvenirs
E. Go for a water/restroom break
G. Go to the child-friendly rides
H. Go to the Haunted House
I. Go to the Simulator rides
J. Sightseeing
K. Go to the Spinning rides"""

df = pd.read_csv(file_path, usecols=[column_name])
df

,Arrange the following activities in the order in which you experienced them during your visit.\nA. Go to the rollercoasters\nB. Go to the Water-based rides\nC. Go to the F&B outlet\nD. Go and shop for souvenirs\nE. Go for a water/restroom break\nG. Go to the child-friendly rides\nH. Go to the Haunted House\nI. Go to the Simulator rides\nJ. Sightseeing\nK. Go to the Spinning rides
0,E A D B D E I C
1,J G H I E B C A K D
2,H E K J I E B
3,A B C E H I G K J D
4,A E I C J D K G B H
...,...
154,J A I C E B G D
155,J A I C E B G D
156,A I C J E B G D
157,A I J C E B G D


### Drop rows with "NA"

In [24]:
df = df.dropna()

In [25]:

# Remove all occurrences of the letter 'F' and 'H' from each entry in the specified column
df[column_name] = df[column_name].str.replace('F ', '', regex=True)
df[column_name] = df[column_name].str.replace('H ', '', regex=True)
df.head()  # Display the DataFrame to confirm changes


/var/folders/s3/x_gq7dtj25z5d_3nmhjk3d340000gn/T/ipykernel_28066/287367131.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.replace('F ', '', regex=True)
/var/folders/s3/x_gq7dtj25z5d_3nmhjk3d340000gn/T/ipykernel_28066/287367131.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.replace('H ', '', regex=True)


,Arrange the following activities in the order in which you experienced them during your visit.\nA. Go to the rollercoasters\nB. Go to the Water-based rides\nC. Go to the F&B outlet\nD. Go and shop for souvenirs\nE. Go for a water/restroom break\nG. Go to the child-friendly rides\nH. Go to the Haunted House\nI. Go to the Simulator rides\nJ. Sightseeing\nK. Go to the Spinning rides
0,E A D B D E I C
1,J G I E B C A K D
2,E K J I E B
3,A B C E I G K J D
4,A E I C J D K G B H


In [26]:
# Rename the column
df = df.rename(columns={column_name: 'path'})
# Export DataFrame to CSV file
df.to_csv('../data/itinerary.csv', index=False)  # index=False to avoid writing row numbers

### Load the leaned DataFrame into SQLite

In [27]:
conn = sqlite3.connect(':memory:')  # Use an in-memory SQLite database
df.to_sql('activities', conn, index=False, if_exists='replace')

157

### Step 4: Count occurrences of each letter and calculate percentages

In [28]:
query = """
WITH letter_counts AS (
    SELECT
        SUBSTR(activities.[{}], 1, 1) AS letter,
        COUNT(*) AS count
    FROM activities
    GROUP BY letter
),
total_counts AS (
    SELECT SUM(count) AS total
    FROM letter_counts
)
SELECT
    letter_counts.letter,
    letter_counts.count,
    (letter_counts.count * 100.0 / total_counts.total) AS percentage
FROM letter_counts, total_counts
ORDER BY letter_counts.letter;
""".format(column_name)

# Execute the query and retrieve the results
result = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

result

DatabaseError: Execution failed on sql '
WITH letter_counts AS (
    SELECT
        SUBSTR(activities.[Arrange the following activities in the order in which you experienced them during your visit.
A. Go to the rollercoasters
B. Go to the Water-based rides
C. Go to the F&B outlet
D. Go and shop for souvenirs
E. Go for a water/restroom break
G. Go to the child-friendly rides
H. Go to the Haunted House
I. Go to the Simulator rides
J. Sightseeing
K. Go to the Spinning rides], 1, 1) AS letter,
        COUNT(*) AS count
    FROM activities
    GROUP BY letter
),
total_counts AS (
    SELECT SUM(count) AS total
    FROM letter_counts
)
SELECT
    letter_counts.letter,
    letter_counts.count,
    (letter_counts.count * 100.0 / total_counts.total) AS percentage
FROM letter_counts, total_counts
ORDER BY letter_counts.letter;
': no such column: activities.Arrange the following activities in the order in which you experienced them during your visit.
A. Go to the rollercoasters
B. Go to the Water

### Checking if the percentage column sums up to 100%

In [ ]:
# Calculate the sum of the percentage column
total_percentage = result['percentage'].sum()

# Display the result
total_percentage

# if you want the total % column to sum up to 100% (but not necessary), you can 
# do so by rounding the percentage column to 2 decimal places

# result['percentage'] = result['percentage'].round(2)

# # Recalculate the sum to verify
# total_percentage = result['percentage'].sum()
# total_percentage

99.99999999999999

### Putting the resulting output into a new csv file

In [ ]:
# Specify the file path for your CSV output
output_file_path = 'activity_counts_and_popularity_percentages.csv'

# Save the DataFrame to a CSV file
result.to_csv(output_file_path, index=False)

print(f"Data has been successfully saved to {output_file_path}")


Data has been successfully saved to activity_counts_and_popularity_percentages.csv
